In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_excel("Data/tr.xlsx",)

In [2]:
over_missing_sample = []
row_value_count = train_data.apply(pd.Series.value_counts,axis=1,dropna=False)

In [126]:
import tools
#Data transform
train_y = pd.DataFrame(train_data['outcome']) 
train_X = pd.DataFrame(train_data.drop(['outcome'],axis=1))

train_X = tools.data_transform(train_X)

In [127]:
import tools
tools.set_pandas_display_options()
train_X.describe()

,AGE,SEX,LOS,Joint,Drain,Cemented,Commercial_ALBC,Non_commercial_ALBC,cci_index,elx_index,Blood_trans,ASA,CBC_WBC,CBC_RBC,CBC_HG,CBC_HT,CBC_MCV,CBC_MCH,CBC_MCHC,CBC_RDW,CBC_Platelet,CBC_RDWCV,BUN,Crea,GOT,GPT,ALB,Na,K,UA,Diagnosis,Congestive Heart Failure,Cardiac Arrhythmia,Valvular Disease,Heart disease,Pulmonary Circulation Disorders,Peripheral Vascular Disorders,Hypertension Uncomplicated,Paralysis,Other Neurological Disorders,Chronic Pulmonary Disease,Lung disease,Diabetes,Hypothyroidism,Renal Failure,Liver Disease,Peptic Ulcer Disease excluding bleeding,AIDS/HIV,Lymphoma,Metastatic Cancer,Solid Tumor without Metastasis,Cancer history,Rheumatoid Arthritis/collagen,Coagulopathy,Obesity,Weight Loss,Fluid and Electrolyte Disorders,Blood Loss Anemia,Deficiency Anemia,Anemia,Alcohol Abuse,Drug Abuse,Psychoses,Depression,Psyciatric disorder,OP_time
count,52065.000000,52065.000000,52065.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,32001.000000,16408.000000,16380.000000,24734.000000,24617.000000,16359.000000,16372.000000,16369.000000,16364.000000,16554.000000,3595.000000,24102.000000,22129.000000,22877.000000,17847.000000,15268.000000,15894.000000,16166.000000,12717.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,52159.000000,32053.000000
mean,65.303620,0.331874,5.619456,1.328726,0.739086,0.671524,0.092103,0.583178,0.701490,0.938975,0.141433,2.414425,8.233603,5.657630,13.687125,38.873498,90.645497,30.863361,34.343660,21.280638,241.175178,14.694887,18.611407,1.761761,30.133059,27.044994,5.027797,141.864696,4.971970,6.767256,0.396307,0.023888,0.035679,0.021875,0.081443,0.002627,0.006346,0.274929,0.002569,0.014283,0.057056,0.059683,0.119078,0.010813,0.028432,0.066470,0.055887,0.000403,0.001802,0.003624,0.039686,0.045112,0.043502,0.006844,0.011829,0.001936,0.015472,0.003758,0.009068,0.012826,0.011848,0.001150,0.003068,0.018099,0.021166,255.064175
std,12.214685,0.470890,2.891653,0.469755,0.439137,0.469664,0.289174,0.493038,1.345236,1.429831,0.348471,0.545605,2.405581,0.544212,1.819272,5.162323,7.014657,2.743574,1.093986,12.650694,68.958212,1.198110,7.466866,0.858865,104.882890,22.373833,0.768206,3.176008,0.742038,1.815913,0.945208,0.152703,0.185491,0.146278,0.327844,0.051183,0.079409,0.446483,0.050621,0.118657,0.231952,0.244860,0.323884,0.103423,0.166206,0.249104,0.229705,0.020061,0.042414,0.060087,0.195223,0.224930,0.203985,0.082448,0.108118,0.043962,0.123421,0.061186,0.094796,0.115883,0.108205,0.033897,0.055301,0.133309,0.151594,81.919669
min,12.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.170000,3.060000,1.190000,19.800000,48.110000,12.420000,25.130000,12.340000,16.100000,12.500000,1.800000,0.280000,4.000000,2.000000,2.000000,22.100000,2.300000,1.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
25%,59.000000,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,6.870000,5.320000,12.490000,35.400000,88.610000,30.120000,33.730000,14.040000,199.000000,13.960000,14.270000,1.030000,21.000000,16.000000,4.350000,140.320000,4.300000,5.540000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [128]:
threshhold =len(train_X.columns)*0.25
over_missing = row_value_count[np.nan]<=threshhold
class_0 = train_y['outcome']==1

print(sorted(train_y.value_counts()))
train_X = train_X[over_missing|class_0]
train_y = train_y[over_missing|class_0]
print(sorted(train_y.value_counts()))


[879, 51280]
[879, 27076]


In [129]:
# Data split
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(
    train_X ,
    train_y,
    test_size=0.2,
    random_state=42)

print(sorted(train_y.value_counts()))
print(sorted(val_y.value_counts()))

[718, 21646]
[161, 5430]


In [130]:
from imblearn.under_sampling import RandomUnderSampler as RUS

print(sorted(train_y.value_counts()))

rus = RUS(sampling_strategy=0.1,random_state=42)
train_X,train_y = rus.fit_resample(train_X,train_y)

print(sorted(train_y.value_counts()))

[718, 21646]
[718, 7180]


In [131]:
# Missing value imputation
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import tools
feature_kind = tools.init_feature_kind(train_X)
cont,cate = tools.get_feature_kind(train_X,feature_kind)  

strategy = 'median'

imp_mean = IterativeImputer(max_iter=100,random_state=0)
imp_mean.fit(train_X[cont])

train_X[cont] = imp_mean.transform(train_X[cont])
val_X[cont] = imp_mean.transform(val_X[cont])

print("filled continuous missing value with "+strategy)

strategy = 'constant'

imp = SimpleImputer(missing_values=np.nan, strategy=strategy,fill_value=10.0)
imp.fit(train_X[cate])

train_X[cate] = imp.transform(train_X[cate])
val_X[cate] = imp.transform(val_X[cate])


print("filled nominal missing value with ",strategy)


filled continuous missing value with median
filled nominal missing value with  constant


In [132]:
from sklearn import preprocessing


cont,cate = tools.get_feature_kind(train_X,feature_kind)

scaler = preprocessing.StandardScaler().fit(train_X[cont])
train_X[cont] = scaler.transform(train_X[cont])
val_X[cont] = scaler.transform(val_X[cont])

In [133]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
print(enc.fit_transform(train_y))


  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 0)	1.0
  (12, 0)	1.0
  (13, 0)	1.0
  (14, 0)	1.0
  (15, 0)	1.0
  (16, 0)	1.0
  (17, 0)	1.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 0)	1.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 0)	1.0
  (24, 0)	1.0
  :	:
  (7873, 1)	1.0
  (7874, 1)	1.0
  (7875, 1)	1.0
  (7876, 1)	1.0
  (7877, 1)	1.0
  (7878, 1)	1.0
  (7879, 1)	1.0
  (7880, 1)	1.0
  (7881, 1)	1.0
  (7882, 1)	1.0
  (7883, 1)	1.0
  (7884, 1)	1.0
  (7885, 1)	1.0
  (7886, 1)	1.0
  (7887, 1)	1.0
  (7888, 1)	1.0
  (7889, 1)	1.0
  (7890, 1)	1.0
  (7891, 1)	1.0
  (7892, 1)	1.0
  (7893, 1)	1.0
  (7894, 1)	1.0
  (7895, 1)	1.0
  (7896, 1)	1.0
  (7897, 1)	1.0


In [134]:
from torch.utils.data import dataloader
import torch



device = torch.device('cuda')

# create feature and targets tensor for train set. As you remember we need variable to accumulate gradients. Therefore first we create tensor, then we will create variable
tensor_train_X = torch.from_numpy(np.array(train_X)).to(device)
tensor_train_y = torch.from_numpy(np.array(train_y)).to(device)# data type is long


# create feature and targets tensor for val set.
tensor_val_X = torch.from_numpy(np.array(val_X)).to(device)
tensor_val_y = torch.from_numpy(np.array(val_y)).to(device) # data type is long


# Pytorch train and val sets
train = torch.utils.data.TensorDataset(tensor_train_X, tensor_train_y)
val = torch.utils.data.TensorDataset(tensor_val_X, tensor_val_y)

In [135]:
import torch.nn as nn
import torch.nn.functional as F

class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_dim,100)
        self.relu1 = nn.Sigmoid()
        self.fc2 = nn.Linear(100,40)
        self.relu2 = nn.Sigmoid()
        self.out = nn.Linear(input_dim,output_dim)
        self.softmax = nn.Softmax(1)
            
    def forward(self, x):
        
       # out = self.fc1(x)
      #  out = self.relu1(out)
      #  out = self.fc2(out)
      #  out = self.relu2(out)
        out = self.out(x)
        
        out = self.softmax(out)
        return out



In [136]:
from torch.utils.data import WeightedRandomSampler
# Hyper parm
# batch_size, epoch and iteration
batch_size = 100
n_iters = 11000
num_epochs = n_iters / (len(train_X) / batch_size)
num_epochs = int(num_epochs)



# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle = False)
model = ANNModel(train_X.shape[1],int((train_X.shape[1]+train_y.shape[1])/2),1).to(device)

# Cross Entropy Loss 
error = nn.CrossEntropyLoss(weight=torch.tensor([0.1,1]))

# SGD Optimizer
learning_rate = 0.1
optimizer = torch.optim.Adam(model.parameters())
lr_scheduler = torch.optim.lr_scheduler.StepLR( optimizer, 
                                                step_size = 50, 
                                                gamma = 0.1)

In [137]:
from torch.autograd import Variable
# ANN model training
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

model.train()

performace = {}


for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        

        optimizer.zero_grad() # Clear gradients
        outputs = model(Variable(X.float())) # Forward propagation
        #print(outputs)
        print(y.float())
        loss = error(outputs,y.float()) # Calculate softmax and cross entropy loss
        loss.backward() # Calculating gradients
        optimizer.step() # Update parameters
        
        count += 1
        
        if count % 50 == 0:
            # Calculate Accuracy         
            predict = []
            true = []
            model.eval()
            # Predict val dataset
            for X, y in val_loader:
                #print(X.float())
                outputs = model(X.float()) # Forward propagation
                for pred in outputs:
                    if(int(pred)>0.5):
                        predict.append(1)
                    else:
                        predict.append(0)
                for label in y:
                    true.append(int(label))
            performace['count'] = (tools.get_performance(true,predict))
            
            if count % 1000 == 0:
                print(count)
                print(performace['count'])
                
            model.train()
    

tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
      

RuntimeError: cross_entropy: weight tensor should be defined either for all 1 classes or no classes but got weight tensor of shape: [2]